# Content Based Movie Recommendation

In [35]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import metrics
from scipy.sparse import csr_matrix

### Import Dataset

In [36]:
# Import movie dataset (combined metadata), subset the data to 10k rows for computational conviencies
whole_df = pd.read_csv('combined_metadata_table.csv')

### Pick A Movie (Fake Search Engine)

In [37]:
def identify_movie(your_pick, whole_df):
    return whole_df[whole_df['title'].str.contains(your_pick, flags=re.IGNORECASE, regex=True)]

In [38]:
your_pick = 'Jumanji'
identify_movie(your_pick, whole_df)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics,id,overview,popularity,revenue,tagline
15078,tt0113497,Jumanji,Jumanji,1995,2/16/96,"Adventure, Comedy, Family",104,USA,"English, French",Joe Johnston,"Jonathan Hensleigh, Greg Taylor",TriStar Pictures,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,7.0,278445,39.0,220.0,101.0,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249.0,Roll the dice and unleash the excitement!


### Subset Dataset

In [39]:
def subset_by_genre(your_pick, whole_df):
    genres = whole_df[whole_df['title'] == your_pick]['genre']
    genres = list(genres)[0].split(', ')
    masks = []
    for genre in genres:
        mask = whole_df['genre'].str.contains(genre)
        masks.append(mask)
    mask = masks[0]
    for i in range(1,len(masks)):
        mask = mask | masks[i]
    return whole_df[mask]

In [40]:
def subset_by_year(your_pick, whole_df, year_range=30):
    year = whole_df[whole_df['title'] == your_pick]['year']
    year = list(year)[0]
    window = [year-year_range, year+year_range]
    mask = (whole_df['year'] >= window[0]) & (whole_df['year'] <= window[1])
    return whole_df[mask]

In [41]:
subset_df = subset_by_genre(your_pick, whole_df)
source = subset_by_year(your_pick, subset_df)

In [42]:
source.head(3)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics,id,overview,popularity,revenue,tagline
5889,tt0059126,Dr. Who and the Daleks,Dr. Who and the Daleks,1965,8/23/65,"Adventure, Family, Sci-Fi",82,UK,English,Gordon Flemyng,"Terry Nation, Milton Subotsky",AARU Productions,"Peter Cushing, Roy Castle, Jennie Linden, Robe...",An eccentric inventor and his companions trave...,5.8,3497,NaN,63.0,36.0,26581,Scientist Doctor Who accidentally activates hi...,2.187942,0.0,Now on the Big Screen in COLOUR!
5891,tt0059269,A High Wind in Jamaica,A High Wind in Jamaica,1965,6/4/65,"Adventure, Drama",103,UK,"English, Spanish",Alexander Mackendrick,"Stanley Mann, Ronald Harwood",Twentieth Century-Fox Productions,"Anthony Quinn, James Coburn, Dennis Price, Lil...","In 1870, a Jamaican colonial family sends its ...",6.6,1716,NaN,31.0,18.0,38340,"In 1870, a Jamaican colonial family sends its ...",1.235903,0.0,NaN
5892,tt0059014,Carry On Cowboy,Carry On Cowboy,1965,5/27/66,"Comedy, Western",93,UK,English,Gerald Thomas,Talbot Rothwell,Peter Rogers Productions,"Sidney James, Kenneth Williams, Jim Dale, Char...",Stodge City is in the grip of the Rumpo Kid an...,6.2,2463,NaN,24.0,8.0,5065,Stodge City is in the grip of the Rumpo Kid an...,5.236409,0.0,How the west was lost!


### Data Processing

In [43]:
# Delete spaces in names to make names more unique
# eg. 'firstname lastname' -> 'firstnamelastname'
column_with_names = ['director', 'writer', 'production_company', 'actors']
source = source.copy()
for col in column_with_names:
    source[str(col)] = source[str(col)].str.replace(' ', '', regex=True)
    source[str(col)] = source[str(col)].str.replace(',', ' ', regex=True)

### Compute TFIDF

In [44]:
# Suggestion: remove the spaces for the names, to solving clustering ambiguity
# Issue: how to deal with numbers ???
columns = ['country', 'language', 'director',
          'writer', 'production_company', 'actors', 'description',
          'overview', 'tagline']

In [45]:
# Converts the source dataframe into single string for tfidf computation
df = {}
movies = []
titles = []
imdbid = []
for i in range(source.shape[0]):
    row = source.iloc[i]
    row_str = ''
    titles.append(row['title'])
    imdbid.append(row['imdb_title_id'])
    for column in columns:
        row_str += str(row[column])
        row_str += ' '
    movies.append(row_str)

df['IMDBid'] = imdbid
df['Title'] = titles
df['Content'] = movies
df = pd.DataFrame(df)

In [46]:
df.head()

,IMDBid,Title,Content
0,tt0059126,Dr. Who and the Daleks,UK English GordonFlemyng TerryNation MiltonSub...
1,tt0059269,A High Wind in Jamaica,"UK English, Spanish AlexanderMackendrick Stanl..."
2,tt0059014,Carry On Cowboy,UK English GeraldThomas TalbotRothwell PeterRo...
3,tt0059800,Thunderball,"UK English, French TerenceYoung RichardMaibaum..."
4,tt0059362,The Knack ...and How to Get It,UK English RichardLester CharlesWood AnnJellic...


In [47]:
# Tranform to tfidf space
V = TfidfVectorizer()
X = V.fit_transform(df['Content'])

### Compute Similarity

In [48]:
# Compute similarity of movie: Melvin and Howard
index = df[df['Title'] == your_pick].index[0]
d1 = list(csr_matrix.toarray(X[index]))
mag_d1 = np.linalg.norm(d1)
dist = []
for i in range(X.shape[0]):
    row = list(csr_matrix.toarray(X[i]))
    dot_product_xy = np.multiply(d1, row).sum(1)
    mag_row = np.linalg.norm(row)
    x_time_y = mag_d1 * mag_row
    dist.append(dot_product_xy/x_time_y) 
dist_series = pd.Series(dist)
dist_series = dist_series.sort_values(ascending=False)
dist_series.iloc[1:6]
dist_series = pd.DataFrame(dist_series)

### Recommend Top 5 Movies

In [49]:
# Merge similarity scores with original dataframe to visualize
result = pd.merge(dist_series, df, how='inner', left_index=True, right_index=True)

In [50]:
# Show closest results
result.head(6)

,0,IMDBid,Title,Content
4270,[1.0],tt0113497,Jumanji,"USA English, French JoeJohnston JonathanHensle..."
10702,[0.2273118975939273],tt2229842,Table No.21,India Hindi AdityaDatt SheershakAnand Shantanu...
7043,[0.1652077041341472],tt0406375,Zathura: A Space Adventure,USA English JonFavreau ChrisVanAllsburg DavidK...
10028,[0.15808931012590421],tt1772341,Wreck-It Ralph,USA English RichMoore RichMoore PhilJohnston W...
4237,[0.15798048398964504],tt0109327,Brainscan,"UK, Canada, USA English JohnFlynn BrianOwens A..."
10548,[0.15278878676723232],tt2294965,Zero Charisma,USA English KatieGraham AndrewMatthews AndrewM...


### Some Ideas for Future

In [17]:
# make a window of year range
# take the difference between year to see the closest
# make it fast: compute the tfidf once per day, put in cache